In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.Network
import jline.lang.ClosedClass
import jline.lang.RoutingMatrix
import jline.VerboseLevel
import jline.lang.constant.ReplacementStrategy
import jline.lang.nodes.Delay
import jline.lang.nodes.Cache
import jline.lang.processes.Exp
import jline.lang.processes.Immediate
import jline.lang.processes.Zipf
import jline.solvers.mva.MVA
import jline.solvers.ssa.SSA
import jline.solvers.nc.NC
GlobalConstants.setVerbose(VerboseLevel.STD);

In [2]:
val nItems = 1000
val cacheSize = 50
// Create the model
val model = Network("Cache Model")

// Block 1: Create nodes
val clientDelay = Delay(model, "Client")
val cacheNode = Cache(model, "Cache", nItems, cacheSize, ReplacementStrategy.LRU)
val cacheDelay = Delay(model, "CacheDelay")

In [3]:
// Block 2: Create job classes
val clientClass = ClosedClass(model, "ClientClass", 1, clientDelay, 0)
val hitClass = ClosedClass(model, "HitClass", 0, clientDelay, 0)
val missClass = ClosedClass(model, "MissClass", 0, clientDelay, 0)

// Configure service times
clientDelay.setService(clientClass, Immediate())  // Immediate request generation
cacheDelay.setService(hitClass, Exp.fitMean(0.2))  // Fast service for hits
cacheDelay.setService(missClass, Exp.fitMean(1.0))  // Slow service for misses

// Configure cache access pattern and behavior
cacheNode.setRead(clientClass, Zipf(1.4, nItems))  // Zipf distribution for realistic access
cacheNode.setHitClass(clientClass, hitClass)     // Route hits to hitClass
cacheNode.setMissClass(clientClass, missClass)   // Route misses to missClass

In [4]:
// Block 3: Configure network topology
val P = model.initRoutingMatrix()

// Routing from client to cache
P.set(clientClass, clientClass, clientDelay, cacheNode, 1.0)

// Routing out of the cache (hits and misses go to cache delay)
P.set(hitClass, hitClass, cacheNode, cacheDelay, 1.0)
P.set(missClass, missClass, cacheNode, cacheDelay, 1.0)

// Return to the client after service
P.set(hitClass, clientClass, cacheDelay, clientDelay, 1.0)
P.set(missClass, clientClass, cacheDelay, clientDelay, 1.0)

// Link the routing matrix to the model
model.link(P)

In [5]:
// Block 4: Solve
SSA(model,"samples",20000).getAvgTable().print()

SSA samples:   20000
SSA analysis [method: default(serial), lang: java, env: 17.0.14] completed in 13.217358s. Iterations: 20000.
Station     JobClass     QLen        Util        RespT       ResidT      ArvR        Tput        
-------------------------------------------------------------------------------------------------
Client      ClientClass  0           0           0           0           0           2.93425     
CacheDelay  HitClass     0.48405     0.48405     0.20000     0.16505     0           2.42026     
CacheDelay  MissClass    0.51595     0.51595     1.00000     0.17476     0           0.51595     
-------------------------------------------------------------------------------------------------
